# Import Libraries

Import Necessary Libraries. Here I will be using Keras library for making classifier. We will also requires numpy and Image for making image to array implementation. We will be using sequential model here in the notebook. 

In [ ]:
#Importing Necessary Libraries.
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [ ]:
import os
print(os.listdir("../input"))

# Data Preperation

* Data Preperation: We will create two list image_arr and labels. image_arr will hold all images in array format while labels will contain their corresponding labels (0,1).*

**Note:** Infected images belongs to class 1, and uninfected images belongs to class 0

In [ ]:
image_arr = []
labels = []

infected = os.listdir("../input/cell_images/cell_images/Parasitized/")
uninfected = os.listdir("../input/cell_images/cell_images/Uninfected/")

for img in infected:
    try:
        image = cv2.imread("../input/cell_images/cell_images/Parasitized/" + img)
        image_from_array = Image.fromarray(image, 'RGB')  #Read in the image with PIL image function in colour mode.
        resize_img = image_from_array.resize((60, 60))  #Resize the image to 60 * 60
        image_arr.append(np.array(resize_img))
        labels.append(1)
        
    except AttributeError:
        print("An error occured while reading in the image")

for img in uninfected:
    try:
        image=cv2.imread("../input/cell_images/cell_images/Uninfected/" + img)
        image_from_array = Image.fromarray(image, 'RGB')
        resize_img = image_from_array.resize((60, 60))
        image_arr.append(np.array(resize_img))
        labels.append(0)
        
    except AttributeError:
        print("An error occur while reading the image")
    

In [ ]:
print(len(image_arr))
print(len(labels))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image_arr[0])
plt.title("Sample of uninfected malaria cell")

Saving the processed images as numpy file

In [ ]:
plt.imshow(image_arr[-1])
plt.title("Sample of infected malaria cell")

In [ ]:
import random

indx = list(range(0, len(labels)))
random.shuffle(indx)

image_arr_sh = []
labels_sh = []

for i in indx:
    image_arr_sh.append(image_arr[i])
    labels_sh.append(labels[i])

In [ ]:
np.save("image_arr", image_arr)
np.save("labels", labels)

In [ ]:
image_arr = np.load("image_arr.npy")
labels = np.load("labels.npy")

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img

Xtrain, Xval, ytrain, yval = train_test_split(image_arr, labels, test_size=0.2)

batch_size = 32
ntrain = len(Xtrain)
nval = len(Xval)

In [ ]:
#Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(Xtrain, ytrain, batch_size=batch_size, shuffle=False)
val_generator = train_datagen.flow(Xval, yval, batch_size=batch_size, shuffle=False)

 # Create Keras Model

In [ ]:
#creating sequential model
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(60,60,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1,activation="sigmoid"))#2 represent output layer neurons 
model.summary()

In [ ]:
# compile the model with loss as categorical_crossentropy and using adam optimizer you can test result by trying RMSProp as well as Momentum
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Fit the model with min batch size as 50[can tune batch size to some factor of 2^power ] 
model.fit_generator(train_generator,steps_per_epoch=ntrain//batch_size,epochs=10,
                             validation_data=val_generator, validation_steps=nval//batch_size)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


# predict probabilities for test set
yhat_probs = model.predict(Xval, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(Xval, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(yval, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(yval, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(yval, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(yval, yhat_classes)
print('F1 score: %f' % f1)

# confusion matrix
matrix = confusion_matrix(yval, yhat_classes)
print(matrix)

#classiication report
cp = classification_report(yval, yhat_classes)
print(cp)

In [ ]:
import seaborn as sb
import pandas as pd

columns = ['infected','uninfected']  

cp_df = pd.DataFrame(matrix,columns,columns)                      
plt.figure(figsize=(10,6))  
sb.heatmap(cp_df, annot=True)

In [ ]:

#Fit the model with min batch size as 50[can tune batch size to some factor of 2^power ] 
history = model.fit_generator(train_generator,steps_per_epoch=ntrain//batch_size,epochs=10,
                             validation_data=val_generator, validation_steps=nval//batch_size)

# Plot of loss and validation accuracy

In [ ]:
print(history.history.keys())

In [ ]:
#lets plot the train and val curve
#get the details form the history object
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

In [ ]:
from keras.models import load_model
model.save('malaria_model.h5')
model.save_weights('malaria_model_wieghts.h5')

# Use of Model

In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image
from PIL import Image
import numpy as np
import os
import cv2
def convert_to_array(img):
    im = cv2.imread(img)
    img_ = Image.fromarray(im, 'RGB')
    image = img_.resize((50, 50))
    return np.array(image)
def get_cell_name(label):
    if label==0:
        return "Paracitized"
    if label==1:
        return "Uninfected"
def predict_cell(file):
    model = load_model('cells.h5')
    print("Predicting Type of Cell Image.................................")
    ar=convert_to_array(file)
    ar=ar/255
    label=1
    a=[]
    a.append(ar)
    a=np.array(a)
    score=model.predict(a,verbose=1)
    print(score)
    label_index=np.argmax(score)
    print(label_index)
    acc=np.max(score)
    Cell=get_cell_name(label_index)
    return Cell,"The predicted Cell is a "+Cell+" with accuracy =    "+str(acc)

